This lab on Decision Trees in R is an abbreviated version of p. 324-331 of "Introduction to Statistical Learning with
Applications in R" by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani.

# 8.3.1 Fitting Classification Trees

The ${\tt tree}$ library is useful for constructing classification and regression trees:

In [ ]:
library(tree)
library(ISLR)

We'll start by using **classification trees** to analyze the ${\tt Carseats}$ data set. In these
data, ${\tt Sales}$ is a continuous variable, and so we begin by converting it to a
binary variable. We use the ${\tt ifelse()}$ function to create a variable, called
${\tt High}$, which takes on a value of ${\tt Yes}$ if the ${\tt Sales}$ variable exceeds 8, and
takes on a value of ${\tt No}$ otherwise:

In [ ]:
High=ifelse(Carseats$Sales<=8,"No","Yes")

Then we'll use the ${\tt data.frame()}$ function to merge ${\tt High}$ with the rest of
the ${\tt Carseats}$ data:

In [ ]:
Carseats=data.frame(Carseats,High)
head(Carseats)

In order to properly evaluate the performance of a classification tree on
the data, we must estimate the test error rather than simply computing
the training error. We first split the observations into a training set and a test
set:

In [ ]:
set.seed(2)
train=sample(1:nrow(Carseats), 200)
Carseats.test=Carseats[-train,]
High.test=High[-train]

We now use the ${\tt tree()}$ function to fit a classification tree in order to predict
${\tt High}$ using all variables but ${\tt Sales}$ (that would be a little silly...). The syntax of the ${\tt tree()}$ function is quite
similar to that of the ${\tt lm()}$ function:

In [ ]:
tree.carseats=tree(High~.-Sales,Carseats,subset=train)

The ${\tt summary()}$ function lists the variables that are used as internal nodes (forming decision points)
in the tree, the number of terminal nodes, and the (training) error rate:

In [ ]:
summary(tree.carseats)

We see that the training error rate is 10.5%. For classification trees, the ${\tt deviance}$
reported in the output of ${\tt summary()}$ is given by:

$$-2\sum_m\sum_k n_{mk}log\hat{p}_{mk}$$

where $n_{mk}$ is the number of observations in the $m^{th}$ terminal node that
belong to the $k^{th}$ class. A small ${\tt deviance}$ indicates a tree that provides
a good fit to the (training) data. The ${\tt residual \ mean \ deviance}$ reported is
simply the ${\tt deviance}$ divided by $n−|T_0|$.

One of the most attractive properties of trees is that they can be
graphically displayed. We use the ${\tt plot()}$ function to display the tree structure,
and the ${\tt text()}$ function to display the node labels. The argument
${\tt pretty=0}$ instructs ${\tt R}$ to include the category names for any qualitative predictors,
rather than simply displaying a letter for each category:

In [ ]:
plot(tree.carseats)
text(tree.carseats,pretty=0)

The most important indicator of ${\tt High}$ sales appears to be shelving location,
since the first branch differentiates ${\tt Good}$ locations from ${\tt Bad}$ and ${\tt Medium}$
locations.

If we just type the name of the tree object, ${\tt R}$ prints output corresponding
to each branch of the tree. ${\tt R}$ displays the split criterion (e.g. ${\tt Price}<142$), the
number of observations in that branch, the deviance, the overall prediction
for the branch (${\tt Yes}$ or ${\tt No}$), and the fraction of observations in that branch
that take on values of ${\tt Yes}$ and ${\tt No}$. Branches that lead to terminal nodes are
indicated using asterisks:

In [ ]:
tree.carseats

Finally, let's evaluate the tree's performance on
the test data. The ${\tt predict()}$ function can be used for this purpose. In the
case of a classification tree, the argument ${\tt type="class"}$ instructs ${\tt R}$ to return
the actual class prediction. This approach leads to correct predictions for
around 71.5% of the test data set:

In [ ]:
tree.pred=predict(tree.carseats,Carseats.test,type="class")
table(tree.pred,High.test)
# (86+57)/200 = 71.5%

# Pruning

Next, we consider whether **pruning** the tree might lead to improved
results. The function ${\tt cv.tree()}$ performs cross-validation in order to
determine the optimal level of tree complexity; cost complexity pruning
is used in order to select a sequence of trees for consideration. We use
the argument ${\tt FUN=prune.misclass}$ in order to indicate that we want the
**classification error rate** as our cost function to guide the cross-validation and pruning process,
rather than the default for the ${\tt cv.tree()}$ function, which is ${\tt deviance}$. The
${\tt cv.tree()}$ function reports the number of terminal nodes of each tree considered
(size) as well as the corresponding error rate and the value of the
cost-complexity parameter used ($k$, which corresponds to $\alpha$ in the equation we saw in lecture).

In [ ]:
set.seed(3)
cv.carseats=cv.tree(tree.carseats,FUN=prune.misclass)

Note that, despite the name, the ${\tt dev}$ field corresponds to the **cross-validation error
rate** in this instance. Let's plot the error
rate as a function of size:

In [ ]:
plot(cv.carseats$size,cv.carseats$dev,type="b")

We see from this plot that the tree with 9 terminal nodes results in the lowest
cross-validation error rate, with 50 cross-validation errors. 

We now apply the ${\tt prune.misclass()}$ function in order to prune the tree to
obtain the nine-node tree by setting the parameter ${\tt best=9}$:

In [ ]:
prune.carseats=prune.misclass(tree.carseats,best=9)
plot(prune.carseats)
text(prune.carseats,pretty=0)

How well does this pruned tree perform on the test data set? Once again,
we can apply the ${\tt predict()}$ function top find out:

In [ ]:
tree.pred=predict(prune.carseats,Carseats.test,type="class")
table(tree.pred,High.test)

Now $\frac{(94+60)}{200} =$ 77% of the test observations are correctly classified, so not only has
the pruning process produced a more interpretable tree, but it has also
improved the classification accuracy.

If we increase the value of ${\tt best}$, we obtain a larger pruned tree...

In [ ]:
prune.carseats=prune.misclass(tree.carseats,best=15)
plot(prune.carseats)
text(prune.carseats,pretty=0)

...with lower
classification accuracy!

In [ ]:
tree.pred=predict(prune.carseats,Carseats.test,type="class")
table(tree.pred,High.test)
# (86+62)/200 = 74%

# 8.3.2 Fitting Regression Trees

Now let's try fitting a **regression tree** to the ${\tt Boston}$ data set from the ${\tt MASS}$ library. First, we create a
training set, and fit the tree to the training data using ${\tt medv}$ (median home value) as our response:

In [ ]:
library(MASS)
set.seed(1)
train = sample(1:nrow(Boston), nrow(Boston)/2)
tree.boston=tree(medv~.,Boston,subset=train)
summary(tree.boston)

Notice that the output of ${\tt summary()}$ indicates that only three of the variables
have been used in constructing the tree. In the context of a regression
tree, the ${\tt deviance}$ is simply the sum of squared errors for the tree. Let's
plot the tree:

In [ ]:
plot(tree.boston)
text(tree.boston,pretty=0)

The variable ${\tt lstat}$ measures the percentage of individuals with lower
socioeconomic status. The tree indicates that lower values of ${\tt lstat}$ correspond
to more expensive houses. The tree predicts a median house price
of \$46,380 for larger homes (${\tt rm}>=7.437$) in suburbs in which residents have high socioeconomic
status (${\tt lstat}<9.715$).

Now we use the ${\tt cv.tree()}$ function to see whether pruning the tree will
improve performance:

In [ ]:
cv.boston=cv.tree(tree.boston)
plot(cv.boston$size,cv.boston$dev,type='b')

The 7-node tree is selected by cross-validation. We can prune the tree using the
${\tt prune.tree()}$ function as before:

In [ ]:
prune.boston=prune.tree(tree.boston,best=7)
plot(prune.boston)
text(prune.boston,pretty=0)

Now we'll use the pruned tree to make predictions on the test set:

In [ ]:
yhat=predict(prune.boston,newdata=Boston[-train,])
boston.test=Boston[-train,"medv"]
plot(yhat,boston.test)
abline(0,1)
mean((yhat-boston.test)^2)

In other words, the test set MSE associated with the regression tree is
25.72. The square root of the MSE is therefore around 5.07, indicating
that this model leads to test predictions that are within around \$5,070 of
the true median home value for the suburb.
    
# 8.3.3 Bagging and Random Forests

Let's see if we can improve on this result using **bagging** and **random forests**. The exact results obtained in this section may
depend on the version of ${\tt R}$ and the version of the ${\tt randomForest}$ package
installed on your computer, so don't stress out if you don't match up exactly with the book. Recall that **bagging** is simply a special case of
a **random forest** with $m = p$. Therefore, the ${\tt randomForest()}$ function can
be used to perform both random forests and bagging. Let's start with bagging:

In [ ]:
library(randomForest)
set.seed(1)
bag.boston=randomForest(medv~.,data=Boston,subset=train,mtry=13,importance=TRUE)
bag.boston

The argument ${\tt mtry=13}$ indicates that all 13 predictors should be considered
for each split of the tree -- in other words, that bagging should be done. How
well does this bagged model perform on the test set?

In [ ]:
yhat.bag = predict(bag.boston,newdata=Boston[-train,])
plot(yhat.bag, boston.test)
abline(0,1)
mean((yhat.bag-boston.test)^2)

The test setMSE associated with the bagged regression tree is just over
half that obtained using an optimally-pruned single tree! We can change
the number of trees grown by ${\tt randomForest()}$ using the ${\tt ntree}$ argument:

In [ ]:
bag.boston=randomForest(medv~.,data=Boston,subset=train,mtry=13,ntree=25)
yhat.bag = predict(bag.boston,newdata=Boston[-train,])
mean((yhat.bag-boston.test)^2)

We can grow a random forest in exactly the same way, except that
we'll use a smaller value of the ${\tt mtry}$ argument. By default, ${\tt randomForest()}$
uses $p/3$ variables when building a random forest of regression trees, and
$\sqrt{p}$ variables when building a random forest of classification trees. Here we'll
use ${\tt mtry = 6}$:

In [ ]:
set.seed(1)
rf.boston=randomForest(medv~.,data=Boston,subset=train,mtry=6,importance=TRUE)
yhat.rf = predict(rf.boston,newdata=Boston[-train,])
mean((yhat.rf-boston.test)^2)

The test set MSE is even lower; this indicates that random forests yielded an
improvement over bagging in this case.

Using the ${\tt importance()}$ function, we can view the importance of each
variable:

In [ ]:
importance(rf.boston)

Two measures of variable importance are reported. The former is based
upon the **mean decrease of accuracy in predictions** on the out-of-bag samples
when a given variable is excluded from the model. The latter is a measure
of the **total decrease in node impurity** that results from splits over that
variable, averaged over all tree. In the
case of regression trees, the node impurity is measured by the training
RSS, and for classification trees by the deviance. Plots of these importance
measures can be produced using the ${\tt varImpPlot()}$ function:

In [ ]:
varImpPlot(rf.boston)

The results indicate that across all of the trees considered in the random
forest, the wealth level of the community (${\tt lstat}$) and the house size (${\tt rm}$)
are by far the two most important variables.

# 8.3.4 Boosting

Now we'll use the ${\tt gbm}$ package, and within it the ${\tt gbm()}$ function, to fit **boosted
regression trees** to the ${\tt Boston}$ data set. We run ${\tt gbm()}$ with the option
${\tt distribution="gaussian"}$ since this is a regression problem; if it were a binary
classification problem, we would use ${\tt distribution="bernoulli"}$. The
argument ${\tt n.trees=5000}$ indicates that we want 5000 trees, and the option
${\tt interaction.depth=4}$ limits the depth of each tree:

In [ ]:
library(gbm)
set.seed(1)
boost.boston=gbm(medv~.,data=Boston[train,],distribution="gaussian",n.trees=5000,interaction.depth=4)

The ${\tt summary()}$ function produces a relative influence plot and also outputs
the relative influence statistics:

In [ ]:
summary(boost.boston)

We see that ${\tt lstat}$ and ${\tt rm}$ are again the most important variables by far. We can
also produce partial dependence plots for these two variables. These plots
illustrate the marginal effect of the selected variables on the response after
integrating out the other variables. In this case, as we might expect, median
house prices are increasing with ${\tt rm}$ and decreasing with ${\tt lstat}$:

In [ ]:
par(mfrow=c(1,2))
plot(boost.boston,i="rm")
plot(boost.boston,i="lstat")

Now let's use the boosted model to predict ${\tt medv}$ on the test set:

In [ ]:
yhat.boost=predict(boost.boston, newdata=Boston[-train,], n.trees=5000)
mean((yhat.boost-boston.test)^2)

The test MSE obtained is similar to the test MSE for random forests
and superior to that for bagging. If we want to, we can perform boosting
with a different value of the shrinkage parameter $\lambda$. The default
value is 0.001, but this is easily modified. Here we take $\lambda = 0.2$:

In [ ]:
boost.boston=gbm(medv~., data=Boston[train,], distribution="gaussian", n.trees=5000, interaction.depth=4, shrinkage=0.2, verbose=F)
yhat.boost=predict(boost.boston, newdata=Boston[-train,], n.trees=5000)
mean((yhat.boost-boston.test)^2)

In this case, using $\lambda = 0.2$ leads to a slightly lower test MSE than $\lambda = 0.001$.

To get credit for this lab, post your responses to the following questions:
 - What's one real-world scenario where you might try using Bagging?
 - What's one real-world scenario where you might try using Random Forests?
 - What's one real-world scenario where you might try using Boosting?
 
to Piazza: https://piazza.com/class/igwiv4w3ctb6rg?cid=53